# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments          Customer  \
0  Dec 30 2022  1:16PM  254072        19      8954898  Eywa Pharma Inc.   
1  Dec 30 2022  1:16PM  254072        19      8955460  Eywa Pharma Inc.   
2  Dec 30 2022  1:16PM  254072        19      8959691  Eywa Pharma Inc.   
3  Dec 30 2022  1:16PM  254072        19      8959695  Eywa Pharma Inc.   
4  Dec 30 2022 12:42PM  254070        15      8968315             Lonza   
5  Dec 30 2022 12:39PM  254069        10  Enova-11154       Emerginnova   
6  Dec 30 2022 12:33PM  254068        10   8868355-BO  Eywa Pharma Inc.   
7  Dec 30 2022 12:33PM  254068        10   8868357-BO  Eywa Pharma Inc.   
8  Dec 30 2022 12:33PM  254068        10   8878244-BO  Eywa Pharma Inc.   
9  Dec 30 2022 12:33PM  254068        10   8897499-BO  Eywa Pharma Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6      Executing  
7      Executing  
8      Executing  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
17  254068      Executing          4
18  254068       Released          3
19  254069       Released          1
20  254070       Released          1
21  254072       Released          4

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
254066                1.0       NaN
254068                4.0       3.0
254069                NaN       1.0
254070                NaN       1.0
254072                NaN       4.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
254016                0.0       3.0
254027                0.0       1.0
254031                0.0       1.0
254039                0.0       1.0
254042                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
254016                  0         3
254027                  0         1
254031                  0         1
254039                  0         1
254042                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               254016          0         3
1               254027          0         1
2               254031          0         1
3               254039          0         1
4               254042          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               254016                  3
1               254027                  1
2               254031                  1
3               254039                  1
4               254042                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Dec 30 2022  1:16PM  254072        19                Eywa Pharma Inc.
4   Dec 30 2022 12:42PM  254070        15                           Lonza
5   Dec 30 2022 12:39PM  254069        10                     Emerginnova
6   Dec 30 2022 12:33PM  254068        10                Eywa Pharma Inc.
13  Dec 30 2022 12:30PM  254066        12               Zelenko Labs, LLC
14  Dec 30 2022 12:23PM  254063        10                Eywa Pharma Inc.
21  Dec 30 2022 12:21PM  254062        16  Sartorius Bioprocess Solutions
22  Dec 30 2022 11:32AM  254057        19                  Innogenix, LLC
45  Dec 30 2022 11:18AM  254055        10                     Emerginnova
46  Dec 30 2022 10:53AM  254050        10                Eywa Pharma Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Dec 30 2022  1:16PM  254072        19                Eywa Pharma Inc.   
1  Dec 30 2022 12:42PM  254070        15                           Lonza   
2  Dec 30 2022 12:39PM  254069        10                     Emerginnova   
3  Dec 30 2022 12:33PM  254068        10                Eywa Pharma Inc.   
4  Dec 30 2022 12:30PM  254066        12               Zelenko Labs, LLC   
5  Dec 30 2022 12:23PM  254063        10                Eywa Pharma Inc.   
6  Dec 30 2022 12:21PM  254062        16  Sartorius Bioprocess Solutions   
7  Dec 30 2022 11:32AM  254057        19                  Innogenix, LLC   
8  Dec 30 2022 11:18AM  254055        10                     Emerginnova   
9  Dec 30 2022 10:53AM  254050        10                Eywa Pharma Inc.   

  Executing Released  
0                  4  
1                  1  
2                  1  
3         4        3  
4         1           
5         6        1  
6                  1  
7                 23  
8                  1  
9                  3

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Dec 30 2022  1:16PM  254072        19                Eywa Pharma Inc.   
1  Dec 30 2022 12:42PM  254070        15                           Lonza   
2  Dec 30 2022 12:39PM  254069        10                     Emerginnova   
3  Dec 30 2022 12:33PM  254068        10                Eywa Pharma Inc.   
4  Dec 30 2022 12:30PM  254066        12               Zelenko Labs, LLC   
5  Dec 30 2022 12:23PM  254063        10                Eywa Pharma Inc.   
6  Dec 30 2022 12:21PM  254062        16  Sartorius Bioprocess Solutions   
7  Dec 30 2022 11:32AM  254057        19                  Innogenix, LLC   
8  Dec 30 2022 11:18AM  254055        10                     Emerginnova   
9  Dec 30 2022 10:53AM  254050        10                Eywa Pharma Inc.   

  Released Executing  
0        4            
1        1            
2        1            
3        3         4  
4                  1  
5        1         6  
6        1            
7       23            
8        1            
9        3

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released Executing
0  Dec 30 2022  1:16PM  254072        19   Eywa Pharma Inc.        4          
1  Dec 30 2022 12:42PM  254070        15              Lonza        1          
2  Dec 30 2022 12:39PM  254069        10        Emerginnova        1          
3  Dec 30 2022 12:33PM  254068        10   Eywa Pharma Inc.        3         4
4  Dec 30 2022 12:30PM  254066        12  Zelenko Labs, LLC                  1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse           Customer  Released  \
0  Dec 30 2022  1:16PM  254072        19   Eywa Pharma Inc.       4.0   
1  Dec 30 2022 12:42PM  254070        15              Lonza       1.0   
2  Dec 30 2022 12:39PM  254069        10        Emerginnova       1.0   
3  Dec 30 2022 12:33PM  254068        10   Eywa Pharma Inc.       3.0   
4  Dec 30 2022 12:30PM  254066        12  Zelenko Labs, LLC       NaN   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        4.0  
4        1.0

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Dec 30 2022  1:16PM  254072        19   Eywa Pharma Inc.       4.0   
1  Dec 30 2022 12:42PM  254070        15              Lonza       1.0   
2  Dec 30 2022 12:39PM  254069        10        Emerginnova       1.0   
3  Dec 30 2022 12:33PM  254068        10   Eywa Pharma Inc.       3.0   
4  Dec 30 2022 12:30PM  254066        12  Zelenko Labs, LLC       0.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        4.0  
4        1.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3048562      27.0       10.0
12          762164       2.0        1.0
15          254070       1.0        0.0
16          508104       2.0        0.0
19          508129      27.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3048562      27.0       10.0
1        12   762164       2.0        1.0
2        15   254070       1.0        0.0
3        16   508104       2.0        0.0
4        19   508129      27.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      27.0       10.0
1        12       2.0        1.0
2        15       1.0        0.0
3        16       2.0        0.0
4        19      27.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   27.0
1        12  Released    2.0
2        15  Released    1.0
3        16  Released    2.0
4        19  Released   27.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   15   16    19
Status                              
Executing  10.0  1.0  0.0  0.0   0.0
Released   27.0  2.0  1.0  2.0  27.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   15   16    19
0          Executing  10.0  1.0  0.0  0.0   0.0
1           Released  27.0  2.0  1.0  2.0  27.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   15   16    19
0  Executing  10.0  1.0  0.0  0.0   0.0
1   Released  27.0  2.0  1.0  2.0  27.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()